In [7]:
from fastai.vision import *

In [8]:
DATA = untar_data(URLs.MNIST_SAMPLE)
src = (ImageList.from_folder(DATA, convert_mode='L')
                .split_by_folder(valid='valid')
                .label_from_folder())
data = (src.databunch(bs=8, num_workers=0)
           .normalize((tensor(0.128), tensor(0.305))))
data

ImageDataBunch;

Train: LabelList (12396 items)
x: ImageList
Image (1, 28, 28),Image (1, 28, 28),Image (1, 28, 28),Image (1, 28, 28),Image (1, 28, 28)
y: CategoryList
3,3,3,3,3
Path: /home/user/.fastai/data/mnist_sample;

Valid: LabelList (2038 items)
x: ImageList
Image (1, 28, 28),Image (1, 28, 28),Image (1, 28, 28),Image (1, 28, 28),Image (1, 28, 28)
y: CategoryList
3,3,3,3,3
Path: /home/user/.fastai/data/mnist_sample;

Test: None

In [9]:
def get_mdl(actn:Callable):
    layers = [conv2d(1, 16, stride=2), actn(), conv2d(16, 32, stride=2), actn(), AdaptiveConcatPool2d(1), Flatten(), nn.Linear(64, data.c)]
    return nn.Sequential(*layers)

In [10]:
from mish_cuda import *
mdl_mish = get_mdl(MishCuda)
mdl_mish

Sequential(
  (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (1): MishCuda()
  (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (3): MishCuda()
  (4): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (5): Flatten()
  (6): Linear(in_features=64, out_features=2, bias=True)
)

In [11]:
lrn_mish = Learner(data, mdl_mish, metrics=[accuracy])
cbs = []

In [12]:
lrn_mish.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.184366,0.161153,0.951914,00:06
1,0.079940,0.074344,0.973503,00:06
2,0.033569,0.042023,0.986752,00:06
3,0.045685,0.036185,0.986752,00:06
4,0.037368,0.037822,0.985770,00:06
5,0.017288,0.028351,0.989205,00:06
6,0.015544,0.025071,0.992640,00:06
7,0.008614,0.034780,0.987733,00:06
8,0.011460,0.023918,0.994112,00:06
9,0.006774,0.024139,0.994112,00:06
